In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load the dataset
df = pd.read_csv('institute_most_recent_processed.csv')

# Select relevant columns for analysis
key_columns = [
    'INSTNM', 'STABBR',  # For display: Name and State
    'TUITIONFEE_IN', 'TUITION_GAP', 'AFFORDABILITY_INDEX', 'COSTT4_A',  # Affordability
    'ADM_RATE', 'DIVERSITY_SCORE', 'UGDS',  # Access
    'GRAD_RATE', 'RET_FT4', 'MD_EARN_WNE_P10', 'QUALITY_SCORE',  # Outcomes
    'IS_PUBLIC'  # Contextual
]

# Drop rows with missing values in key metrics (focus on non-null for scoring)
df_clean = df[key_columns].dropna(subset=[
    'TUITIONFEE_IN', 'TUITION_GAP', 'AFFORDABILITY_INDEX', 'COSTT4_A',
    'ADM_RATE', 'DIVERSITY_SCORE',
    'GRAD_RATE', 'RET_FT4', 'MD_EARN_WNE_P10', 'QUALITY_SCORE'
])

print(f"Clean dataset shape: {df_clean.shape}")

# Initialize scaler
scaler = MinMaxScaler()

# Normalize each metric to 0-1
metrics = df_clean.select_dtypes(include=[np.number]).columns
df_normalized = pd.DataFrame(scaler.fit_transform(df_clean[metrics]), 
                             columns=metrics, index=df_clean.index)

# Invert lower-better metrics for Affordability (higher score = better)
lower_better = ['TUITIONFEE_IN', 'TUITION_GAP', 'COSTT4_A']
for col in lower_better:
    if col in df_normalized.columns:
        df_normalized[col] = 1 - df_normalized[col]

# Note: AFFORDABILITY_INDEX is already higher-better, no inversion needed

# Compute sub-scores (simple average of normalized metrics)
# Affordability: Average of cost-related (inverted where needed)
aff_cols = ['TUITIONFEE_IN', 'TUITION_GAP', 'AFFORDABILITY_INDEX', 'COSTT4_A']
df_normalized['Affordability_Score'] = df_normalized[aff_cols].mean(axis=1)

# Access: Average of ADM_RATE (higher=better), DIVERSITY_SCORE (higher=better)
# Use UGDS as tiebreaker? Not in score, but for sorting if needed
access_cols = ['ADM_RATE', 'DIVERSITY_SCORE']
df_normalized['Access_Score'] = df_normalized[access_cols].mean(axis=1)

# Outcomes: Average of GRAD_RATE, RET_FT4, MD_EARN_WNE_P10, QUALITY_SCORE (all higher=better)
outcomes_cols = ['GRAD_RATE', 'RET_FT4', 'MD_EARN_WNE_P10', 'QUALITY_SCORE']
df_normalized['Outcomes_Score'] = df_normalized[outcomes_cols].mean(axis=1)

# Overall Score: Equal weight average of sub-scores
df_normalized['Overall_Score'] = df_normalized[['Affordability_Score', 'Access_Score', 'Outcomes_Score']].mean(axis=1)

# Merge back with names and state for display
df_final = pd.concat([df_clean[['INSTNM', 'STABBR', 'IS_PUBLIC']], df_normalized[['Affordability_Score', 'Access_Score', 'Outcomes_Score', 'Overall_Score']]], axis=1)

# Rank by Overall Score descending (top = best)
df_ranked = df_final.sort_values('Overall_Score', ascending=False).reset_index(drop=True)

# Display top 10
top_10 = df_ranked.head(10)[['INSTNM', 'STABBR', 'Affordability_Score', 'Access_Score', 'Outcomes_Score', 'Overall_Score']]
print(top_10.to_string(index=False))

# Optional: Save to CSV
df_ranked.to_csv('college_rankings.csv', index=False)

Clean dataset shape: (5650, 14)
                                   INSTNM STABBR  Affordability_Score  Access_Score  Outcomes_Score  Overall_Score
                Samuel Merritt University     CA             0.653684      0.806633        0.655198       0.705171
           Chamberlain University-Florida     FL             0.609262      0.838151        0.608753       0.685389
    Roseman University of Health Sciences     NV             0.653684      0.797838        0.590655       0.680726
   American University of Health Sciences     CA             0.635233      0.875736        0.516385       0.675785
                    The King's University     TX             0.676285      0.882338        0.466287       0.674970
          Chamberlain University-Virginia     VA             0.650194      0.860534        0.513015       0.674581
SUNY Downstate Health Sciences University     NY             0.653684      0.807632        0.561113       0.674143
                       Brazosport College     TX